<a href="https://colab.research.google.com/github/EtzionR/NLP4GeoAI/blob/main/preprocess_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --force-reinstall datasets==2.18.0

  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.2 kB)
  Using cached pyarrow_hotfix-0.7-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.6.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.18-py312-none-any.whl.metadata (7.5 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



In [ ]:
from datasets import load_dataset
ontonotes = load_dataset("tner/ontonotes5")
ontonotes

In [ ]:
from datasets import load_dataset

conll = load_dataset("eriktks/conll2003")
conll

In [ ]:
def dataset_to_pandas(ds):
    return pd.concat([ds['train'].to_pandas(),
                      ds['validation'].to_pandas(),
                      ds['test'].to_pandas()])

cdf = dataset_to_pandas(conll)
cdf

In [ ]:
odf = dataset_to_pandas(ontonotes)
odf

In [ ]:
values, counts = np.unique([word for sentence in odf.tokens for word in sentence if len(word)<4], return_counts=1)

index = np.argsort(counts)[::-1]



STOPWORDS = {"'s", "n't",  "'re","'m", "'ll", "'ve","'ll",
             '"', '``', "''",'$',';', '%',
             '?','/?', ':', '--',
             '-',
             ', ',
             '/.', '.'}

values[index][80:100]

In [ ]:
[' '.join(word_list).replace(' .','.').replace(' ,',',') for word_list in ontonotes['train']['tokens']]

In [ ]:
[' '.join(word_list).replace(' .','.').replace(' ,',',') for word_list in conll['train']['tokens']]